In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
load_dotenv()
import warnings
warnings.filterwarnings("ignore")
 
api_key = os.getenv('OPENAI_API_KEY')
model = AzureChatOpenAI(temperature=0, deployment_name="chat")



In [2]:


from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.document_loaders import PyPDFLoader
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document = PyPDFLoader("../Resources/GCP.pdf")
data = document.load()
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        length_function = len,
        chunk_overlap = 250
)

documents = text_splitter.split_documents(data)
db = Chroma.from_documents(documents, embedding_function)
chain_qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="map_reduce",
    retriever = db.as_retriever(search_kwargs={"k": 5})
)


In [3]:
chain_qa.invoke("Qué es BigQuery")

{'query': 'Qué es BigQuery',
 'result': 'BigQuery es un servicio de almacenamiento y análisis de datos en la nube ofrecido por Google Cloud. Permite a los usuarios almacenar grandes volúmenes de datos estructurados y no estructurados, y realizar consultas y análisis rápidos sobre ellos utilizando SQL. BigQuery es altamente escalable y puede manejar conjuntos de datos de varios terabytes o incluso petabytes. También ofrece características avanzadas como el procesamiento paralelo masivo y la capacidad de integrarse con otras herramientas y servicios de Google Cloud. Además, BigQuery también ofrece capacidades de aprendizaje automático a través de BigQuery ML, lo que permite a los usuarios crear y entrenar modelos de aprendizaje automático directamente en los datos almacenados en BigQuery.'}